### Import necessary libraries

In [1]:
import os
import random
import string
import time
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding, utils
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.exceptions import InvalidSignature

### Function to generate RSA key pair

In [2]:
def generate_rsa_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

### Function for RSA encryption

In [3]:
def rsa_encrypt(message, public_key):
    start_time = time.time()
    
    # Split the message into blocks (in practice, handle this more robustly)
    block_size = 128
    blocks = [message[i:i+block_size] for i in range(0, len(message), block_size)]

    # Encrypt each block
    encrypted_blocks = [public_key.encrypt(
        block,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    ) for block in blocks]

    encrypted_message = b''.join(encrypted_blocks)

    end_time = time.time()
    print(f"RSA Encryption Time taken: {end_time - start_time} seconds")

    return encrypted_message

### Function for RSA decryption

In [4]:
def rsa_decrypt(encrypted_message, private_key):
    start_time = time.time()

    # Split the encrypted message into blocks
    block_size = 256
    encrypted_blocks = [encrypted_message[i:i+block_size] for i in range(0, len(encrypted_message), block_size)]

    # Decrypt each block
    decrypted_blocks = [private_key.decrypt(
        block,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    ) for block in encrypted_blocks]

    decrypted_message = b''.join(decrypted_blocks)

    end_time = time.time()
    print(f"RSA Decryption Time taken: {end_time - start_time} seconds")

    return decrypted_message

- In the code we provided, the block sizes for encryption and decryption are indeed different. The block size for encryption is set to 128 bytes (block_size = 128), while the block size for decryption is set to 256 bytes (block_size = 256). Let's discuss why this might be the case.

- The block size for encryption and decryption in RSA should ideally be the same. The block size is determined by the key size and the padding scheme used. In our code, the padding scheme is padding.OAEP with SHA-256 as the hash function.

- The key size of the RSA key pair affects the block size.

- In practice, it's common to use a padding scheme like OAEP, which adds additional bits to the block for security purposes. The actual block size for encryption and decryption is influenced by the padding scheme and the key size. 

### Randomly Generated 100MB text file

In [5]:
def generate_text_file(file_path, size_in_bytes):
    with open(file_path, 'w') as file:
        # Generate random data and write it to the file
        data = ''.join(random.choices(string.ascii_letters + string.digits, k=size_in_bytes))
        file.write(data)

# Set the file path and size (100 MB)
file_path = 'large_file.txt'
size_in_bytes = 100 * 1024 * 1024  # 100 MB

# Generate the text file
generate_text_file(file_path, size_in_bytes)

print(f"Text file '{file_path}' generated successfully.")

Text file 'large_file.txt' generated successfully.


### Function to read file content

In [6]:
def read_file_content(file_path):
    with open("large_file.txt", 'r') as file:
        return file.read()

### RSA encryption & decryption with the use of  large_text file of 100 mb randomly generated

In [7]:
total_start_time = time.time()

# Specify the path to the input text file
input_file_path = 'large_file.txt'

# Read plaintext message from file
plaintext_message = read_file_content(input_file_path)

# Sender's side
sender_private_key, sender_public_key = generate_rsa_key_pair()

# Step 1: RSA encryption
encrypted_message = rsa_encrypt(plaintext_message.encode(), sender_public_key)

# Write encrypted message to a file
with open('encrypted_output.txt', 'wb') as encrypted_file:
    encrypted_file.write(encrypted_message)

# Receiver's side
# Step 2: RSA decryption
decrypted_message = rsa_decrypt(encrypted_message, sender_private_key)

# Write decrypted message to a file
with open('decrypted_output.txt', 'wb') as decrypted_file:
    decrypted_file.write(decrypted_message)

total_end_time = time.time()
print(f"Total Time taken: {total_end_time - total_start_time} seconds")


RSA Encryption Time taken: 27.779789924621582 seconds
RSA Decryption Time taken: 288.45883679389954 seconds
Total Time taken: 318.2207477092743 seconds


### RSA generate, encrypt, and decrypt a 256-bit key 

In [8]:
# Block 5: Example Usage to generate, encrypt, and decrypt a 256-bit key
total_start_time = time.time()

# Generate a 256-bit key (32 bytes)
aes_key = b'\x01' * 32

# Save AES key to a text file
with open('aes_key.txt', 'wb') as aes_file:
    aes_file.write(aes_key)
    
print(aes_key.hex())


# Sender's side
sender_private_key, sender_public_key = generate_rsa_key_pair()

# Step 1: RSA encryption of the AES key
encrypted_aes_key = rsa_encrypt(aes_key, sender_public_key)
print("Encrypted AES Key:", encrypted_aes_key.hex())

# # Write encrypted AES key to a file
# with open('encrypted_aes_key.txt', 'wb') as encrypted_file:
#     encrypted_file.write(encrypted_aes_key)

# Receiver's side
# Step 2: RSA decryption of the AES key
decrypted_aes_key = rsa_decrypt(encrypted_aes_key, sender_private_key)
print("Decrypted AES Key:", decrypted_aes_key.hex())

total_end_time = time.time()
print(f"Total Time taken: {total_end_time - total_start_time} seconds")

0101010101010101010101010101010101010101010101010101010101010101
RSA Encryption Time taken: 0.0001442432403564453 seconds
Encrypted AES Key: 10d865d0593866b68fca7517991b2fd182843aec33bf890d3081a6747e42f787b7b0ff8824c5a8a485d006a24ff7ef7adde20bff83d5afe1528d9121eeece19c61dbb71c6c6065a1126003491bf9b298ab806d992b89cc78e54bb57a572b10f222ae5cb533c87ebb43a52d316e1314e51cbbb27b9362c58e5e64a299017ef4fd6d36fc3c2ac798c16b0e3c06e26a2b2c01f30c6764a0e624730384768c8d0ab4e4d70fca8609e39a84ec85606b223f135afe04c7c53bff6d7b5968312abf95aef9af1e3f2dc731869695abe65f3610505a2e0c065368f6ce284444cd4352b9c7f57da833bb17e22cbad83391223f8b89fba72b65fa614280c16614817e0e113e
RSA Decryption Time taken: 0.0009937286376953125 seconds
Decrypted AES Key: 0101010101010101010101010101010101010101010101010101010101010101
Total Time taken: 0.24602079391479492 seconds


### AES Encryption, Decryption functions and function to remove padding from the decrypted message

In [9]:
# Function to pad the message to be a multiple of the block size
def pad(message):
    block_size = algorithms.AES.block_size // 8
    padding_length = block_size - (len(message) % block_size)
    padding = bytes([padding_length]) * padding_length
    return message + padding

# Function to remove padding from the decrypted message
def unpad(message):
    padding_length = message[-1]
    return message[:-padding_length]

# AES encryption function
def aes_encrypt(message, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_message = encryptor.update(pad(message)) + encryptor.finalize()
    return encrypted_message

# AES decryption function
def aes_decrypt(encrypted_message, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_message = decryptor.update(encrypted_message) + decryptor.finalize()
    return unpad(decrypted_message)


### Encryption of large_file txt of 100 MB using AES key which we got from earlier block and stored

In [10]:
# Load the AES key from the file
start_time = time.time()
with open('aes_key.txt', 'rb') as key_file:
    aes_key = key_file.read()
end_time = time.time()
print(f"Loading AES key took {end_time - start_time} seconds")

# Specify the path to the input text file
input_file_path = 'large_file.txt'

# Read plaintext message from file
start_time = time.time()
with open('large_file.txt', 'rb') as file:
    plaintext_message = file.read()
end_time = time.time()
print(f"Reading plaintext message took {end_time - start_time} seconds")

# Encrypt the large text file using AES
start_time = time.time()
encrypted_message = aes_encrypt(plaintext_message, aes_key)
end_time = time.time()
print(f"Encrypting message took {end_time - start_time} seconds")

# Write the encrypted message to a file
start_time = time.time()
with open('aes_encrypted_large_file.txt', 'wb') as encrypted_file:
    encrypted_file.write(encrypted_message)
end_time = time.time()
print(f"Writing encrypted message to file took {end_time - start_time} seconds")

# Decrypt the large text file using AES
start_time = time.time()
decrypted_message = aes_decrypt(encrypted_message, aes_key)
end_time = time.time()
print(f"Decrypting message took {end_time - start_time} seconds")

# Write the decrypted message to a file
start_time = time.time()
with open('aes_decrypted_large_file.txt', 'wb') as decrypted_file:
    decrypted_file.write(decrypted_message)
end_time = time.time()
print(f"Writing decrypted message to file took {end_time - start_time} seconds")

Loading AES key took 0.0013799667358398438 seconds
Reading plaintext message took 0.05080461502075195 seconds
Encrypting message took 0.19324469566345215 seconds
Writing encrypted message to file took 1.3886024951934814 seconds
Decrypting message took 0.2001664638519287 seconds
Writing decrypted message to file took 0.7878463268280029 seconds


### RSA Signature Execution Block

In [11]:
# Function to generate RSA signature
def generate_rsa_signature(message, private_key):
    start_time = time.time()

    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

    end_time = time.time()
    print(f"RSA Signature Generation Time taken: {end_time - start_time} seconds")

    return signature

In [12]:
# Function to verify RSA signature
def verify_rsa_signature(message, signature, public_key):
    start_time = time.time()

    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("RSA Signature Verification: Signature is valid")
    except InvalidSignature:
        print("RSA Signature Verification: Signature is invalid")

    end_time = time.time()
    print(f"RSA Signature Verification Time taken: {end_time - start_time} seconds")


In [13]:
# Sender's side
# Step 3: Generate RSA signature
signature = generate_rsa_signature(encrypted_message, sender_private_key)

# Write RSA signature to a file
with open('rsa_signature.txt', 'wb') as signature_file:
    signature_file.write(signature)

# Receiver's side
# Load RSA signature from the file
with open('rsa_signature.txt', 'rb') as signature_file:
    loaded_signature = signature_file.read()

# Step 4: Verify RSA signature
verify_rsa_signature(encrypted_message, loaded_signature, sender_public_key)

RSA Signature Generation Time taken: 0.08523201942443848 seconds
RSA Signature Verification: Signature is valid
RSA Signature Verification Time taken: 0.08192300796508789 seconds
